# DVL - Dragon Version Log V2 

这是一个使用全球整车发版日志进行搜索处理的，以期达到在少量数据上有较高的搜索准确率
v2 使用的数据集是 经过OpenAI创建摘要的发版日志数据集 20240229版本

In [ ]:
!pip install -r '/Users/wangjialong/Documents/code/saic_project/global-vehicle-dragon/data-processing/requirements.txt'

# 加载环境配置

In [2]:
%env ENV_FOR_DYNACONF=local
import os 
os.chdir('/Users/wangjialong/Documents/code/saic_project/global-vehicle-dragon/data-processing')

# 重新加载模块
from importlib import reload
from config.settings import Settings
import db.db_manager
import config.settings
import os
from pymilvus import (
    MilvusClient,
    connections,
    utility,
    FieldSchema, CollectionSchema, DataType,
    Collection,
)
from sqlalchemy import func, create_engine
from towhee import pipe, ops
from db.milvus_manager import MilvusConn
from langchain.text_splitter import MarkdownTextSplitter
import pandas as pd
from db.db_manager import DBConn
from db.model.p_dragon import DVL_V2

reload(config.settings)
reload(db.db_manager)
print(f"Debug: {bool(Settings.DEBUG)}")

env: ENV_FOR_DYNACONF=local
settings loaded:  local


/Users/wangjialong/anaconda3/envs/data-processing/lib/python3.9/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


settings loaded:  local
Debug: False


# 查询源数据目录，加载到DB中

In [3]:
file_directory = r'/Users/wangjialong/Documents/AI/SAIC_DCSGlobalVehicle/DVL/prd_dvl_md_20240229'
files = os.listdir(file_directory)
file_count = sum(os.path.isfile(os.path.join(file_directory, item)) for item in files)
print(f"目录 '{file_directory}' 下有 {file_count} 个文件。")

目录 '/Users/wangjialong/Documents/AI/SAIC_DCSGlobalVehicle/DVL/prd_dvl_md_20240229' 下有 94 个文件。


In [ ]:
from db.model.p_dragon import DVL_V2
import hashlib
from etl.dragon_etl import save_dvl_v2_model
from db.db_manager import DBConn

for index, item in enumerate(files, start=1):
    if item in ['.DS_Store', 'index.md']:
        continue;
    
    db_conn = DBConn()
    full_path = os.path.join(file_directory, item)
    hash_obj = hashlib.sha256()
    if os.path.isfile(full_path):
        print(f"处理文件 {index} |'{item}'")
        try:
            with open(full_path, 'r') as file:
                content = file.read()
                hash_obj.update(content.encode())
                new_model = DVL_V2(
                    name = item,
                    hash = hash_obj.hexdigest(),
                    content = content
                )
            save_dvl_v2_model(conn=db_conn, model=new_model)
        except Exception as e:
            print(f"Failed! {index} | {full_path} | {e}")
    db_conn.close()


In [4]:
from transformers import BertTokenizer, BertModel
import torch
from scipy.spatial.distance import cosine


def sentence_to_vector(sentence):
    """
    定义模型转embedding
    :return embedding
    """
    # 加载分词器和模型
    tokenizer = BertTokenizer.from_pretrained('bert-base-chinese')
    model = BertModel.from_pretrained('bert-base-chinese')
    input = tokenizer(sentence, return_tensors='pt', padding='max_length', max_length=512)
    with torch.no_grad():
        output = model(**input)
    sentence_embedding = output.last_hidden_state.mean(dim=1)
    return sentence_embedding

/Users/wangjialong/anaconda3/envs/data-processing/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 对数据库内的数据进行向量化

使用bert-base-chinese模型，对数据库的content进行分块嵌入到milvus的向量数据库中


In [ ]:
# 参数
database_name = 'dvl_v2'
# 最大文本的长度
max_token = 512
# 查询分区数量
partition_num = 100
# my_model_name = 'shibing624/text2vec-base-chinese'
my_model_name = 'bert-base-chinese'

connections.connect(host=Settings.MILVUS_URL, port=Settings.MILVUS_PORT)
client = MilvusClient(host=Settings.MILVUS_URL, port=Settings.MILVUS_PORT)

print(utility.has_collection(collection_name = database_name))
if utility.has_collection(collection_name = database_name):
    print(f"describe: {client.describe_collection(collection_name = database_name)}")
    print(f"num of entities: {client.num_entities(collection_name = database_name)}")

milvus_conn = MilvusConn(database_name)
db_conn = DBConn()

count_query = f"SELECT count(1) FROM p_dragon.dragon_version_log_v2 LIMIT {partition_num}"
total_num = db_conn.session.query(func.count(DVL_V2.id)).scalar()
iteration_times = total_num // partition_num + (0 if total_num % partition_num == 0 else 1)
print(f"Records Count: {total_num} | iterations time: {iteration_times}")
db_conn.close()

result_pd = pd.DataFrame(columns=['name', 'embedding'])

# 这一块可以抽取相同代码块
for i in range(iteration_times):    
    engine = create_engine(Settings.DB_URL)
    start_num = i * partition_num
    select_query = f"SELECT id, name, content as content FROM p_dragon.dragon_version_log_v2 LIMIT {partition_num} OFFSET {start_num}"     
    with engine.connect() as connection, connection.begin():
        df = pd.read_sql_query(sql=select_query, con=connection.connection)
        for index, row in enumerate(df.itertuples(index=False), start=1):
            if row.id is None:
                break

            if len(row.content) > max_token:
                print(f"{row.id} | {row.name} | {len(row.content)} | 大于最大token长度，切割块")
                markdown_spilitter = MarkdownTextSplitter(chunk_size=max_token, chunk_overlap=0)
                docs = markdown_spilitter.create_documents([row.content])
                for split in docs:
                    normalize_vector = sentence_to_vector(split.page_content)
                    new_row = pd.DataFrame([[row.name, normalize_vector[0].numpy()]],
                        columns=['name', 'embedding'])
                    result_pd = pd.concat([result_pd, new_row])
            else:    
                print(f"处理数据 分片{i}|{index}")
                normalize_vector = sentence_to_vector(row.content)
                new_row = pd.DataFrame([[row.name, normalize_vector[0].numpy()]],
                        columns=['name', 'embedding'])
                result_pd = pd.concat([result_pd, new_row])

        milvus_conn.insert(result_pd)                
        engine.dispose()
milvus_conn.flush()
milvus_conn.stats()
print(f"all done") 
# print(f"data 0 : {df.loc[0]}")

In [6]:
# 根据简单的提示词，看下是否能正确的召回数据
def testify_query(prompt_word):
    embedding = sentence_to_vector(prompt_word)
    # 参数
    database_name = 'dvl_v2'
    milvus_conn = MilvusConn(database_name)
    search_params = {
        "metric_type": "L2", 
        "offset": 0, 
        "ignore_growing": False, 
        "params": {"nprobe": 10}
    }
    milvus_conn.collection.load()
    results = milvus_conn.collection.search(
        data = [embedding[0].numpy()],
        anns_field = 'embedding',
        param = search_params,
        limit = 10,
        expr = None,
        output_fields = ['name'],
        consistency_level = 'Strong'
    )

    # for hit in results[0]:
    # print(f"{hit.entity.get('name')} | {hit.distance}")
    return [hit.entity.get('name') for hit in results[0]]

# 加载验证集

读取验证集文件dvl_qa_20240301.csv

In [57]:
import pandas as pd
from ast import literal_eval

file_dir = './dvl/data/dvl_qa_20240301.csv'
df = pd.read_csv(file_dir)
print(df.shape)
# print(df['question'][0])
# print(df['answer'][0])

(19, 2)


In [58]:
success = []
failed = []

def is_subsequence(s, t):
    """
    检查s是否是t的子序列。
    """
    # print(f"{s} | {t}")
    t_iter = iter(t)
    return all(char in t_iter for char in s)


for idx, item in df.iterrows():
    print(f"progress {idx + 1}")
    answer_list = testify_query(item.question)
    matched = False
    for ret_answer in answer_list:
        if is_subsequence(item.answer, ret_answer):
            matched = True
            break
        else:
            continue

    if matched:
        success.append(item.question)
    else:
        failed.append(item.question)

success_num = len(success)
failed_num = len(failed)
# 打印结果
print(f"Success: {success_num}, Failed: {failed_num}, \
Success Rate : {success_num / (success_num + failed_num):.4f}")

print(f"Success question: {success}")
print(f"Failed question: {failed}")


progress 1


2024-03-01 10:37:33,038 - 140704323120000 - milvus_client.py-milvus_client:553 - DEBUG: Created new connection using: c35817e4784944adb1cb3ba02a3bb38d


progress 2


2024-03-01 10:37:34,971 - 140704323120000 - milvus_client.py-milvus_client:553 - DEBUG: Created new connection using: 26789f79188443369640181c18d44d90


progress 3


...

progress 18


2024-03-01 10:38:08,568 - 140704323120000 - milvus_client.py-milvus_client:553 - DEBUG: Created new connection using: fc95796bd1fb4175ac4ee5140ff4a854


progress 19


2024-03-01 10:38:10,488 - 140704323120000 - milvus_client.py-milvus_client:553 - DEBUG: Created new connection using: 0ceb153b785942f08ad89380b3909e66


Success: 6, Failed: 13, Success Rate : 0.3158
Success question: ['UK By Rule Allocation Scene的配车报告是哪个版本上线的？', '哪些改动涉及ShortageETA的处理', 'SMUK - 配车报告 相关更改', '系统内存监控的改动版本', '南美Invoice query by vin字段更改记录', 'PRD 发版日志 的列表内容']
Failed question: ['我需要一些关于南美Pending Warehouse更改的版本', '哪个改动涉及到了分支\xa0bugfix-20600-GdsShippingAllocationError-20230731', 'BCA相关的改动有哪些版本', 'Gate Out页面是哪个版本新增Download功能的', '英国发票取消后出库凭证取消日期取值错误的issue', '内外饰颜色CODE需要能够支持数字的改动', 'BUG-20600是关于什么问题的修复', '英国区域解决的BUG中，哪个是关于循环调用接口的问题', 'BUG-20591修复了哪个具体问题?', '关于allocate页面和allocate设置页面的shipping detail的bug', 'Global DCS-Vehicle的发版日志模板', 'Offline Vehicles Maintain页面 相关更改', '有关于 Allocate Rule Setting 的修改内容']
